In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from pymongo import MongoClient
import math
import pandas as pd
import numpy as np
import datetime

#client = MongoClient();

In [2]:
speed_threshold = 0.25  # km/sec - Average speed of flight 900 km/hr

In [25]:
class GEO_Map():
	"""
	It hold the  map for zip code and its latitute and longitute
	"""
	__instance = None

	@staticmethod
	def get_instance():
		""" Static access method. """
		if GEO_Map.__instance == None:
			GEO_Map()
		return GEO_Map.__instance

	def __init__(self):
		""" Virtually private constructor. """
		if GEO_Map.__instance != None:
			raise Exception("This class is a singleton!")
		else:
			GEO_Map.__instance = self
			self.map = pd.read_csv("uszipsv.csv", header=None, names=['A',"B",'C','D','E'])
			self.map['A'] =  self.map['A'].astype(str)

	def get_lat(self, pos_id):
		#print("printing value from class ", self.map[self.map.A == pos_id ].B)        
		return self.map[self.map.A == pos_id ].B

	def get_long(self, pos_id):
		return self.map[self.map.A == pos_id ].C

	def distance(self, lat1, long1, lat2, long2):
		theta = long1 - long2
		dist = math.sin(self.deg2rad(lat1)) * math.sin(self.deg2rad(lat2)) + math.cos(self.deg2rad(lat1)) * math.cos(self.deg2rad(lat2)) * math.cos(self.deg2rad(theta))
		dist = math.acos(dist)
		dist = self.rad2deg(dist)
		dist = dist * 60 * 1.1515 * 1.609344
		return dist

	def rad2deg(self, rad):
		return rad * 180.0 / math.pi

	def deg2rad(self, deg):
		return deg * math.pi / 180.0

In [23]:
# Function to check rules for UCL and Credit Score
def verify_ucl_data(card_id, amount):
    #print("into verify_ucl_data function")  
    #print("card id from kafka is ", card_id)
    #print("amount from kafka is ", amount)
    try:
        client = MongoClient();
        # Database Name
        db = client["CREDIT_CARD_DB"]
        
        # Collection Name
        lookupTable = db["lookup_table"]
        
        lookupValue = lookupTable.find_one({'card_id': 378303738095292}) #378303738095292
        #lookupValue = lookupTable.find({'card_id': card_id}) #378303738095292
        #lookupValue = col.find({'member_id': 1314074991813})
        #print("lookupvalue is ", lookupValue)
        #print("lookupvalue type is ", type(lookupValue))
        
        #if lookupValue is None : 
        #    return False
        
        #for data in lookupValue:
            #print("data is ", data)
            #print("dataucl is ", data["ucl"])
            #print("credit score is ", data["score"])
            
        #if data is None : 
        #    return False

        
        if amount < float(lookupValue["ucl"]) and float(lookupValue["ucl"]) > 200:
            return True
        else:
            return False
    except Exception as e:
        raise Exception(e)

In [18]:
"""
Function to verify the following zipcode rules
ZIP code distance
:param card_id: (Long) Card id of the card customer
:param postcode: (Integer) Post code of the card transaction
:param transaction_dt: (String) Timestamp
:return: (Boolean)
"""
def verify_postcode_data(card_id, postcode, transaction_dt):

    try:
        
        client = MongoClient();
        # Database Name
        db = client["CREDIT_CARD_DB"]
        # Collection Name
        lookupTable = db["lookup_table"]
        
        lookupValue = lookupTable.find_one({'card_id': 378303738095292}) #378303738095292
        #print("lookupValue", lookupValue)
        geo_map = GEO_Map.get_instance()
        
        #card_row = hbasedao.get_data(key=str(card_id), table=lookup_table)
        #last_postcode = (card_row[b'card_data:postcode']).decode("utf-8")
        #last_transaction_dt = (card_row[b'card_data:transaction_dt']).decode("utf-8")
        
        last_postcode = lookupValue["postcode"]
        last_transaction_dt = lookupValue["transaction_dt2"]
        #print("last_postcode", last_postcode)
        #print("last_transaction_dt", last_transaction_dt)

        current_lat = geo_map.get_lat(str(postcode))
        for data in current_lat:
            #print("current_lat data is >>>>>>>>>>>>>>>>>>>>", data)
            current_lat1 = data
        current_lon = geo_map.get_long(str(postcode))
        for data in current_lon:
            #print("current_lon data is >>>>>>>>>>>>>>>>>>>>", data)
            current_lon1 = data
        previous_lat = geo_map.get_lat(str(last_postcode))
        for data in previous_lat:
            #print("previous_lat data is >>>>>>>>>>>>>>>>>>>>", data)
            previous_lat1 = data
        previous_lon = geo_map.get_long(str(last_postcode))
        for data in previous_lon:
            #print("previous_lon data is >>>>>>>>>>>>>>>>>>>>", data)
            previous_lon1 = data
        
        #for data in current_lat:
            #print("data is>>>>>>>>>>> ", data)
            #print("dataucl is ", data["ucl"])
            #print("credit score is ", data["score"])
        
        #print("***************************************************************")
        #print("1. current_lat", current_lat1)
        #print("2. current_lon", current_lon1)
        #print("3. previous_lat", previous_lat1)
        #print("4. previous_lon", previous_lon1)
        #print("***************************************************************")
        
        
        dist = geo_map.distance(lat1=current_lat1, long1=current_lon1, lat2=previous_lat1, long2=previous_lon1)
        #print("distance is ************************************************* ", dist)
        #dist = distance2(lat1=current_lat, long1=current_lon, lat2=previous_lat, long2=previous_lon)

        speed = calculate_speed(dist, transaction_dt, last_transaction_dt)
        #print("speed is ****************************************************", speed)

        if speed < speed_threshold:
            return True
        else:
            return False

    except Exception as e:
        raise Exception(e)



In [19]:
"""
A function to calculate the speed from distance and transaction timestamp differentials
:param dist: (Float) Distance between postcodes
:param transaction_dt1: Transaction timestamp from the table
:param transaction_dt2: Transaction timestamp from the POS
:return: (Float) Speed
"""
def calculate_speed(dist, transaction_dt1, transaction_dt2):

    #print("transaction_dt1>>>>>>>>>>>",transaction_dt1)
    #print("transaction_dt2>>>>>>>>>>>",transaction_dt2)
    #transaction_dt1>>>>>>>>>>> 01-03-2018 08:24:29
    #transaction_dt2>>>>>>>>>>> 2018-06-01 14:31:05
    transaction_dt1 = datetime.datetime.strptime(transaction_dt1, '%d-%m-%Y %H:%M:%S')
    transaction_dt2 = datetime.datetime.strptime(transaction_dt2, '%Y-%m-%d %H:%M:%S')

    elapsed_time = transaction_dt1 - transaction_dt2
    elapsed_time = elapsed_time.total_seconds()

    try:
        return dist / elapsed_time
    except ZeroDivisionError:
        return 299792.458  
# (Speed of light)

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]

mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)


In [16]:
"""
A function to update genuine and fraud transactions into the two collections
1. Collection 1 - 
2. Collection 2 - 
"""

client = MongoClient();
# Database Name
mydb = client["CREDIT_CARD_DB"]
mycol = mydb["card_transactions"]
mydict = {"card_id" : 348702330256514, "member_id" : 37495066290, "amount" : 9084849, "postcode" : 33946, "pos_id" : 614677375609919, "transaction_dt" : "11-02-2018 00:00:00", "status" : "GENUINE"}    
mycol.insert_one(mydict)

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]

myquery = { "address": "Valley 345" }
newvalues = { "$set": { "address": "Canyon 123" } }

mycol.update_one(myquery, newvalues)

#print "customers" after the update:
for x in mycol.find():
  print(x)


In [12]:

client = MongoClient();
# Database Name
mydb = client["CREDIT_CARD_DB"]
mycol = mydb["lookup_table"]
lookupUpdQryOld = {"card_id" : 4877961603808028}    
lookupUpdQryNew = { "$set": {"card_id" : 4877961603808028, "member_id" : 15235737327750, "amount" : 1000, "postcode" : 33946, "pos_id" : 614677375609919, "transaction_dt" : "12-02-2021 00:00:00", "status" : "GENUINE"}    }
mycol.update_one(lookupUpdQryOld, lookupUpdQryNew)

In [20]:
def validateFraud(amount, card_id, postcode, txndate):
    #print("into validateFraud function - part1")
    status_ucl_crdScore = verify_ucl_data(card_id, amount)
    #print("Status of UCL and CreditScore is ", status_ucl_crdScore)
    
    status_distance = verify_postcode_data(card_id, postcode, txndate)
    
    if status_ucl_crdScore==True and status_distance==True :
        #print("3 Rules check passed!!!!!! ; congratulations! ")
        return True
    else : 
        return False
    

    #return status_ucl

In [21]:
#{"card_id":348702330256514,"member_id":37495066290,"amount":4380912,"postcode":96774,"pos_id":248063406800722,"transaction_dt":"01-03-2018 08:24:29"}
#validateFraud(kafkajsonObj["amount"], kafkajsonObj["card_id"],kafkajsonObj["postcode"], kafkajsonObj["transaction_dt"])
def validateFraud1(kafkajsonObj):
    #print("into validateFraud function - part1")
    status_ucl_crdScore = verify_ucl_data(kafkajsonObj["card_id"],kafkajsonObj["amount"])
    #print("Status of UCL and CreditScore is ", status_ucl_crdScore)
    
    status_distance = verify_postcode_data(kafkajsonObj["card_id"], kafkajsonObj["postcode"], kafkajsonObj["transaction_dt"])
    
    if status_ucl_crdScore==True and status_distance==True :
        #print("3 Rules check passed!!!!!! ; congratulations! ")
        return True
    else : 
        return False
    

    #return status_ucl

In [28]:
from kafka import KafkaConsumer
import sys

### Setting up the Python consumer
bootstrap_servers = ['18.211.252.152:9092']
topicName = 'transactions-topic-verified'
consumer = KafkaConsumer (topicName, group_id = 'my_group_id7',bootstrap_servers = bootstrap_servers,
auto_offset_reset = 'earliest')   ## You can also set it as latest
i=0
### Reading the message from consumer
try:
    for message in consumer:
        #i=i+1
        #print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,message.offset, message.key,message.value))
        #print ("%s" % (message.value))
        kafkajsonObj = json.loads(message.value)
        status = validateFraud(kafkajsonObj["amount"], kafkajsonObj["card_id"],kafkajsonObj["postcode"], kafkajsonObj["transaction_dt"])
        #status = validateFraud1(kafkajsonObj)
        if (status == False):
            i=i+1
            print("is false? Final status is %d : %s ", i, status)
        
except KeyboardInterrupt:
    sys.exit()

is false? Final status is %d : %s  1 False


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-28-a86ea18a4a5b>", line 17, in <module>
    status = validateFraud(kafkajsonObj["amount"], kafkajsonObj["card_id"],kafkajsonObj["postcode"], kafkajsonObj["transaction_dt"])
  File "<ipython-input-20-2886a5fbe09b>", line 3, in validateFraud
    status_ucl_crdScore = verify_ucl_data(card_id, amount)
  File "<ipython-input-23-1d42bd23bd35>", line 14, in verify_ucl_data
    lookupValue = lookupTable.find_one({'card_id': 378303738095292}) #378303738095292
  File "C:\ProgramData\Anaconda3\lib\site-packages\pymongo\collection.py", line 1459, in find_one
    for result in cursor.limit(-1):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pymongo\cursor.py", line 1248, in next
    if len(self.__data) or self._refresh():
  File "C:\ProgramData\Anaconda3\lib\site-packages\pymongo\cursor.py", line 1165, in _refresh
    self.__send_message(q)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pymongo\cursor.py", line 1052, in __send_messag

TypeError: object of type 'NoneType' has no len()

In [8]:
#Tester Program
status = validateFraud(4380912, 348702330256514,96774, "01-03-2018 08:24:29")
if (status == True):
    print("Genuine")
else:
    print("Fraud")

into validateFraud function - part1
into verify_ucl_data function
card id from kafka is  348702330256514
amount from kafka is  4380912
lookupvalue is  <pymongo.cursor.Cursor object at 0x00000205D9B98100>
lookupvalue type is  <class 'pymongo.cursor.Cursor'>
dataucl is  12794239.889645452
credit score is  225
Status of UCL and CreditScore is  True
lookupValue {'_id': ObjectId('6379a2b6dbd25642124fcf1c'), '': 2, 'card_id': 378303738095292, 'member_id': 1314074991813, 'postcode': 41778, 'transaction_dt2': '2018-06-01 14:31:05', 'ucl': 12794239.889645452, 'score': 225}
last_postcode 41778
last_transaction_dt 2018-06-01 14:31:05
printing value from class  16658    19.9529
Name: B, dtype: float64
current_lat data is >>>>>>>>>>>>>>>>>>>> 19.9529
current_lon data is >>>>>>>>>>>>>>>>>>>> -155.3341
printing value from class  7838    37.267
Name: B, dtype: float64
previous_lat data is >>>>>>>>>>>>>>>>>>>> 37.267
previous_lon data is >>>>>>>>>>>>>>>>>>>> -83.2991
***********************************